In [6]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn 
import sklearn.datasets
import sklearn.linear_model
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder


In [7]:
cancer = load_breast_cancer()


In [8]:
X = cancer.data
y = cancer.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (recommended for neural networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Encode class labels to 0 and 1
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Display the first few rows of the dataset
print("X_train:")
print(X_train[:5])
print("\ny_train:")
print(y_train[:5])

X_train:
[[-1.44075296 -0.43531947 -1.36208497 -1.1391179   0.78057331  0.71892128
   2.82313451 -0.11914956  1.09266219  2.45817261 -0.26380039 -0.01605246
  -0.47041357 -0.47476088  0.83836493  3.25102691  8.43893667  3.39198733
   2.62116574  2.06120787 -1.23286131 -0.47630949 -1.24792009 -0.97396758
   0.72289445  1.18673232  4.67282796  0.9320124   2.09724217  1.88645014]
 [ 1.97409619  1.73302577  2.09167167  1.85197292  1.319843    3.42627493
   2.01311199  2.66503199  2.1270036   1.55839569  0.80531919 -0.81268678
   0.75195659  0.87716951 -0.89605315  1.18122247  0.18362761  0.60059598
  -0.31771686  0.52963649  2.17331385  1.3112795   2.08161691  2.1374055
   0.76192793  3.26560084  1.92862053  2.6989469   1.89116053  2.49783848]
 [-1.39998202 -1.24962228 -1.34520926 -1.10978518 -1.33264483 -0.30735463
  -0.36555756 -0.69650228  1.93033305  0.95437877  0.02752055  1.96305996
  -0.12095781 -0.35077918  0.57276579  0.7394992   0.32065553  0.58946222
   2.61504052  0.71892779 -1

In [9]:
print(X_train.shape)
print(y_train.shape)

(455, 30)
(455,)


In [66]:
X_train = X_train.reshape(X_train.shape[1],X_train.shape[0])
X_test = X_test.reshape(X_test.shape[1],X_test.shape[0])

In [14]:
y_train = y_train.reshape((1, len(y_train)))
y_test = y_test.reshape((1, len(y_test)))

In [79]:
X_train.shape

(30, 455)

In [80]:
y_train.shape

(1, 455)

In [69]:
def sigmoid(x):
    s = 1/(1+np.exp(-x))
    return s

In [70]:
def layer_sizes(X, y):
    
    n_x = X.shape[0]
    n_h = 4
    n_y = y.shape[0]
    return (n_x, n_h, n_y)

In [71]:
def initialize_parameters(n_x, n_h, n_y):
    np.random.seed(2)
    
    w1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h,1))
    w2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    
    assert(w1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(w2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
    
    parameters = {"w1": w1, 
                 "b1": b1,
                 "w2": w2, 
                 "b2": b2}
    return parameters 
    

In [107]:
def forward_propagate(X, parameters):
    w1 = parameters["w1"]
    w2 = parameters["w2"]
    b1 = parameters["b1"]
    b2 = parameters["b2"]
    
    z1 = np.dot(w1, X) + b1
    a1 = np.tanh(z1)
    z2 = np.dot(w2, a1) + b2
    a2 = sigmoid(z2)
    assert(a2.shape == (1,X.shape[1]))
    
    cache = {"z1": z1,
            "a1": a1,
            "z2": z2,
            "a2": a2}
    return a2, cache 

In [97]:
def compute_loss(a2, y, parameters):
    m = y.shape[1]
    logprob = np.multiply(np.log(a2), y) + np.multiply((1-y), np.log(1-a2))
    cost = -1/m * np.sum(logprob)
    
    cost = np.squeeze(cost)
    assert(isinstance(cost, float))
    return cost 

In [98]:
def backward_propagate(parameters, cache, X, y):
    m = X.shape[1]
    
    w1 = parameters["w1"]
    w2 = parameters["w2"]
    
    a1 = cache["a1"]
    a2 = cache["a2"]
    
    dz2 = a2 - y
    dw2 = 1/m * np.dot(dz2, a1.T)
    db2 = 1/m * np.sum(dz2, axis=1, keepdims=True)
    dz1 = np.dot(w2.T, dz2) * (1 - np.power(a1, 2))
    dw1 = 1/m * np.dot(dz1, X.T)
    db1 = 1/m * np.sum(dz1, axis=1, keepdims=True)
    
    grads = {"dw1": dw1, 
            "db1": db1,
            "dw2": dw2,
            "db2": db2}
    return grads 
    
    

In [99]:
def update_parameters(parameters, grads, learning_rate=0.005):
    w1 = parameters["w1"]
    w2 = parameters["w2"]
    b1 = parameters["b1"]
    b2 = parameters["b2"]
    
    dw1 = grads["dw1"]
    dw2 = grads["dw2"]
    db1 = grads["db1"]
    db2 = grads["db2"]
    
    w1 -= learning_rate * dw1
    w2 -= learning_rate * dw2
    b1 -= learning_rate * db1
    b2 -= learning_rate * db2
    
    parameters = {"w1": w1, 
                 "b1": b1,
                 "w2": w2, 
                 "b2": b2}
    return parameters 
    

In [103]:
def nn_model(X, y, num_iterations = 10000, print_cost = False):
    np.random.seed(3)
    n_x = layer_sizes(X, y)[0]
    n_y = layer_sizes(X, y)[2]
    
    n_x, n_h, n_y = layer_sizes(X, y)
    parameters = initialize_parameters(n_x, n_h, n_y)
    w1 = parameters["w1"]
    w2 = parameters["w2"]
    b1 = parameters["b1"]
    b2 = parameters["b2"]
    
    
    for i in range(num_iterations):
        a2, cache = forward_propagate(X, parameters)
        
        cost = compute_loss(a2, y, parameters)
        
        grads = backward_propagate(parameters, cache, X, y)
        
        parameters = update_parameters(parameters, grads)
        
        if i % 1000 == 0:
            print("iterations %i: %f" %(i, cost))
    return parameters 

In [104]:
def predict(parameters, X):
    a2, cache = forward_propagate(X, parameters)
    predictions = np.array([1 if x > 0.5 else 0 for x in a2.reshape(-1, 1)]).reshape(a2.shape)
    return predictions 

In [108]:
parameters = nn_model(X_train, y_train, num_iterations = 10000)
predictions = predict(parameters, X_test)
print("accuracy %d" % float((np.dot(y_test, predictions.T) + np.dot(1-y_test, 1-predictions.T))/ float(y_test.size) * 100) + '%')

iterations 0: 0.693171
iterations 1000: 0.662658
iterations 2000: 0.659981
iterations 3000: 0.659709
iterations 4000: 0.659625
iterations 5000: 0.659451
iterations 6000: 0.658961
iterations 7000: 0.657648
iterations 8000: 0.654595
iterations 9000: 0.649232
accuracy 62%
